## AIDA Practical Quiz
By: Vallarta, Troy Joaquin G.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

source = "https://github.com/Zofserif/aida2-practice/blob/master/datasets/netflix_titles.xlsx?raw=true"

<h2> DATASET READING </h2>

In [2]:
## 1. Read the dataset in xls file.
df = pd.read_excel(source)
df1 = pd.read_excel(source, sheet_name="netflix_titles")
df2 = pd.read_excel(source, sheet_name="netflix_titles_directors")
df3 = pd.read_excel(source, sheet_name="netflix_titles_countries")
df4 = pd.read_excel(source, sheet_name="netflix_titles_cast")
df5 = pd.read_excel(source, sheet_name="netflix_titles_category")

In [3]:
## Getting the categorical data
df5_dummy = pd.get_dummies(df5['listed_in'])
df5_dummy = pd.concat([df5, df5_dummy], axis =1)
df5_dummy.drop(["listed_in"], inplace=True, axis=1)
df5_dummy = df5_dummy.groupby(df5_dummy['show_id']).aggregate("sum")
df5_dummy

,Action & Adventure,Anime Features,Anime Series,British TV Shows,Children & Family Movies,Classic & Cult TV,Classic Movies,Comedies,Crime TV Shows,Cult Movies,...,TV Action & Adventure,TV Comedies,TV Dramas,TV Horror,TV Mysteries,TV Sci-Fi & Fantasy,TV Shows,TV Thrillers,Teen TV Shows,Thrillers
show_id,,,,,,,,,,,,,,,,,,,,,
247747,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
269880,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
281550,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
284890,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292118,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81224868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81227195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81228864,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
## Getting the categorical data
df3_dummy = pd.get_dummies(df3['country'])
df3_dummy = pd.concat([df3, df3_dummy], axis =1)
df3_dummy.drop(["country"], inplace=True, axis=1)
df3_dummy = df3_dummy.groupby(df3_dummy['show_id']).aggregate("sum")
df3_dummy

,Afghanistan,Albania,Argentina,Armenia,Australia,Austria,Azerbaijan,Bangladesh,Belgium,Bermuda,...,United Kingdom,"United Kingdom,",United States,"United States,",Uruguay,Vatican City,Venezuela,Vietnam,West Germany,Zimbabwe
show_id,,,,,,,,,,,,,,,,,,,,,
247747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269880,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
281550,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
284890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292118,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81224868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81227195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81228864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
## 2. Clean the data if not yet cleaned.
df.isnull().sum()

duration_minutes    1969
duration_seasons    4265
type                   1
title                  1
date_added            13
release_year           2
rating                13
description            3
show_id                4
dtype: int64

##### The data is not cleaned.... 

<h2> DATA CLEANING </h2>

In [13]:
# convert data_added from string to date time 
df['date_added'] = pd.to_datetime(df['date_added'])
df['year_added'] = pd.DatetimeIndex(df['date_added']).year
df['month_added'] = df['date_added'].dt.month

ParserError: Unknown string format: This documentary centers on the crew of the B-17 Flying Fortress Memphis Belle as it prepares to execute a strategic bombing mission over Germany.

In [5]:
## 2. Clean the data if not yet cleaned.
netflix_df = df.copy()
netflix_df = df.dropna(subset=['duration_minutes'])
netflix_df.pop("duration_seasons")
nonum = netflix_df[pd.to_numeric(netflix_df['duration_minutes'], errors='coerce').isna()].index
netflix_df = netflix_df.drop(nonum)
netflix_df = netflix_df.astype({'duration_minutes':'int'})
netflix_df.dtypes

duration_minutes      int64
type                 object
title                object
date_added           object
release_year        float64
rating               object
description          object
show_id             float64
dtype: object

In [6]:
def val_missing(x):
  return sum(x.isnull())

#Applying per column:
print ("Missing values per column:")
print (netflix_df.apply(val_missing, axis=0)) 

#Applying per row:
print ("\nMissing values per row:")
print (netflix_df.apply(val_missing, axis=1).head(10)) 

Missing values per column:
duration_minutes    0
type                0
title               0
date_added          2
release_year        1
rating              9
description         1
show_id             2
dtype: int64

Missing values per row:
0     0
1     0
4     0
6     0
7     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [7]:
## 3.  Create a python program that will show the descriptive analysis of the datasets ( Count, min, max, standard deviation, 75%,50%and 25%)
netflix_df.describe().iloc[:,[0,1]]

,duration_minutes,release_year
count,4265.000000,4264.000000
mean,99.100821,2012.370310
std,28.074857,9.679987
min,3.000000,1942.000000
25%,86.000000,2012.000000
50%,98.000000,2016.000000
75%,115.000000,2017.000000
max,312.000000,2020.000000
